In [75]:
import gensim
import string
import re
import pandas as pd
from pymorphy2 import MorphAnalyzer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import sent_tokenize

import warnings
warnings.filterwarnings("ignore")

morph = MorphAnalyzer()

In [2]:
stops = set(stopwords.words('russian'))

In [48]:
path = '/Users/Stoneberry/Desktop/Uni/Мага/dh_books/1933_Belyaev_Pryzhok v nichto.txt'

In [49]:
with open(path, 'r', encoding='utf-8') as f:
    text = f.read()

In [5]:
text[:1000]

'Предисловие К. Э. Циолковского ко второму изданию\n\nОбстоятельный, добросовестный и благоприятный отзыв о романе А. Р. Беляева "Прыжок в ничто" сделан уважаемым проф. Н. А. Рыниным. Этот отзыв в качестве послесловия помещен и в настоящем, втором, издании.\nЯ же могу только подтвердить этот отзыв и прибавить, что из всех известных мне рассказов, оригинальных и переводных, на тему о межпланетных сообщениях роман А. Р. Беляева мне кажется наиболее содержательным и научным. Конечно, возможно лучшее, но, однако, пока его нет. Поэтому я сердечно и искренне приветствую появление второго издания, которое, несомненно, будет способствовать распространению в массах интереса к заатмосферным полетам.\nВероятно, их ожидает великое будущее.\nК. Циолковский\n\nКалуга. Март 1935 г.\nЧасть первая. Акционерное общество спасателей\n\nГлава I\n\nБольшие знания могут быть причиной больших неприятностей\n\n0x01 graphic\nЦандер резко отодвинул чертеж, встал из-за стола, прошелся по кабинету. Вынул из футляр

In [57]:
# 0x01 graphic

## Нормализация

In [71]:
i = 0
all_ = []

for sent in sent_tokenize(text):

    norm_words = []
    words = word_tokenize(sent)
    
    for word in words:
        
        word = word.lower()
        if word.isalpha() and word not in stops:
            norm_words.append(morph.parse(word)[0].normal_form)

    all_.append(norm_words)


In [72]:
len(all_)

7783

## Ngram

In [17]:
from nltk.util import ngrams
from collections import Counter

In [18]:
bigrams = list(ngrams(text_normalized, 2))

In [19]:
bigrams[:10]

[('предисловие', 'циолковский'),
 ('циолковский', 'к'),
 ('к', 'второе'),
 ('второе', 'издание'),
 ('издание', 'обстоятельный'),
 ('обстоятельный', 'добросовестный'),
 ('добросовестный', 'благоприятный'),
 ('благоприятный', 'отзыв'),
 ('отзыв', 'роман'),
 ('роман', 'беляев')]

In [21]:
count = Counter(bigrams)

In [25]:
top100 = sorted(count.items(), key=lambda kv: kv[1], reverse=True)[:100]
top100 = [i[0] for i in top100]

In [26]:
text_normalized += top100

## Topic Modeling

In [28]:
def my_lda(texts, num_topics=10, passes=10, eta='auto'):
            
    dictinary = gensim.corpora.Dictionary(texts)
    dictinary.filter_extremes(no_above=0.3)
    dictinary.compactify()
    
    corpus = [dictinary.doc2bow(text) for text in texts]
 
    lda = gensim.models.LdaModel(corpus, id2word=dictinary, 
                                 passes=passes, num_topics=num_topics,
                                 eta=eta, random_state=42)

    return lda

In [74]:
def words(lda):
    d = {}

    for index, line in enumerate(lda.print_topics()):
        d[index] = re.findall('"(.*?)"', line[1])
    
    return pd.DataFrame(d)

In [73]:
lda = my_lda(all_)

In [76]:
words(lda)

,0,1,2,3,4,5,6,7,8,9
0,голова,это,стормера,земля,каждый,цандёр,человек,очень,леди,говорить
1,нога,ганс,пинч,венера,птица,ответить,конец,словно,хинтон,идти
2,ракета,винклер,сказать,блоттон,свой,пассажир,наш,рука,ракета,дерево
3,время,цандёр,спросить,один,епископ,ракета,почему,ганс,самый,сила
4,ветер,барон,ганс,голос,мэри,ковчег,лицо,текер,земной,ганс
5,фингер,дело,амели,друг,ганс,каюта,тысяча,лететь,скорость,вода
6,работать,дверь,винклер,звездолет,блоттон,мы,цандёр,сторона,камень,работа
7,мочь,видеть,цандёр,найти,скоро,пещера,ганс,воздух,земля,день
8,ганс,время,начать,такой,движение,должный,сидеть,туман,полёт,шар
9,животное,какой,свой,марс,сказать,необходимый,скорый,чёрный,пространство,знать
